In [13]:
import pandas as pd 

df = pd.read_csv('ESK2033.csv')

In [14]:
df = df[['Residual Forecast','RSA Contracted Forecast','Dispatchable Generation','Residual Demand','RSA Contracted Demand']]

In [15]:
df = df.dropna()
df.isnull().sum()

Residual Forecast          0
RSA Contracted Forecast    0
Dispatchable Generation    0
Residual Demand            0
RSA Contracted Demand      0
dtype: int64

In [16]:
df

,Residual Forecast,RSA Contracted Forecast,Dispatchable Generation,Residual Demand,RSA Contracted Demand
2018-04-01 12:00:00 AM,20367.066,20237.000,20237.000,20722.058,1215.902
2018-04-01 01:00:00 AM,19988.733,19744.000,19744.000,20188.493,1203.474
2018-04-01 02:00:00 AM,19731.239,19631.000,19631.000,20019.603,1177.571
2018-04-01 03:00:00 AM,19753.554,19731.000,19731.000,20079.454,1184.312
2018-04-01 04:00:00 AM,19988.365,19890.000,19890.000,20237.490,1197.271
...,...,...,...,...,...
2021-06-07 11:00:00 AM,30268.533,26632.668,26632.668,30314.324,1336.808
2021-06-07 12:00:00 PM,29717.457,26401.624,26401.624,30041.158,1374.298
2021-06-07 01:00:00 PM,29408.647,26126.801,26126.801,29731.365,1333.494
2021-06-07 02:00:00 PM,29455.645,26299.201,26299.201,29727.529,1355.777


In [17]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# define the number of training data points you want to use
n_train = int(0.8 * len(scaled_data))

# scale the data using MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

# split the data into training and testing sets
train_data = scaled_data[:n_train, :]
test_data = scaled_data[n_train:, :]

# prepare the training and testing data for LSTM input
def prepare_data(data, time_steps):
    X, y = [], []
    for i in range(time_steps, len(data)):
        X.append(data[i-time_steps:i, :])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

time_steps = 24   # define the number of past time steps to use for forecasting
X_train, y_train = prepare_data(train_data, time_steps)
X_test, y_test = prepare_data(test_data, time_steps)

NameError: ignored

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

# define the number of training data points you want to use
n_train = int(0.8 * len(df))

# scale the data using MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

# split the data into training and testing sets
train_data = scaled_data[:n_train, :]
test_data = scaled_data[n_train:, :]

# prepare the training and testing data for LSTM input
def prepare_data(data, time_steps):
    X, y = [], []
    for i in range(time_steps, len(data)):
        X.append(data[i-time_steps:i, :])
        y.append(data[i, 3])  # use the column index of Residual Demand (assuming it is the fourth column)
    return np.array(X), np.array(y)

time_steps = 24   # define the number of past time steps to use for forecasting
X_train, y_train = prepare_data(train_data, time_steps)
X_test, y_test = prepare_data(test_data, time_steps)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# define the LSTM model architecture
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))

# compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'mae'])

# fit the LSTM model on the training data
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.1, verbose=1)

# make predictions for a particular month
last_time_steps = df.tail(time_steps).values  # use the last time_steps data points
last_time_steps_scaled = scaler.transform(last_time_steps)
X_pred = np.array([last_time_steps_scaled])  # reshape to match LSTM input shape
y_pred = model.predict(X_pred)

# inverse transform the predicted value to get the actual value in original scale
predicted_demand = scaler.inverse_transform([[0, 0, 0, y_pred[0][0], 0]])[0][3]
print(f"Predicted Residual Demand for the month: {predicted_demand:.2f}")


Epoch 1/50
314/314 [==============================] - 10s 24ms/step - loss: 0.0107 - mse: 0.0107 - mae: 0.0761 - val_loss: 0.0018 - val_mse: 0.0018 - val_mae: 0.0334
Epoch 2/50
314/314 [==============================] - 6s 20ms/step - loss: 0.0014 - mse: 0.0014 - mae: 0.0291 - val_loss: 0.0018 - val_mse: 0.0018 - val_mae: 0.0334
Epoch 3/50
314/314 [==============================] - 10s 32ms/step - loss: 0.0011 - mse: 0.0011 - mae: 0.0254 - val_loss: 0.0019 - val_mse: 0.0019 - val_mae: 0.0368
Epoch 4/50
314/314 [==============================] - 8s 25ms/step - loss: 9.7301e-04 - mse: 9.7301e-04 - mae: 0.0235 - val_loss: 7.5915e-04 - val_mse: 7.5915e-04 - val_mae: 0.0210
Epoch 5/50
314/314 [==============================] - 7s 22ms/step - loss: 8.1401e-04 - mse: 8.1401e-04 - mae: 0.0212 - val_loss: 6.8054e-04 - val_mse: 6.8054e-04 - val_mae: 0.0194
Epoch 6/50
314/314 [==============================] - 7s 21ms/step - loss: 7.9602e-04 - mse: 7.9602e-04 - mae: 0.0211 - val_loss: 6.7655e-04 

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_absolute_error

# evaluate the model on the test data
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# plot the training and validation loss
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# print the MAE and R2 score
print('Test MAE:', mae)
print('Test R2 score:', r2)